# Признаки изображения

Мы увидели, что можно достичь разумной точности в задаче классификации изображений, обучая линейный классификатор на пикселях исходного изображения. В этом задании мы покажем, как улучшить эффективность классификации и обучить классификатор не на пикселях, а на вычисленных с их помощью признаках изображения.

Весь код для задания содержится в этом файле.

In [ ]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt


%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Загружаем данные
Как и в предыдущих упражнениях, загружаем данные CIFAR-10.

In [ ]:
from cs231n.features import color_histogram_hsv, hog_feature

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    # Загружаем датасет
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'

    # Очищаем переменные, чтобы данные можно было загружать много раз 
    try:
       del X_train, y_train
       del X_test, y_test
       print('Очистка предыдущих данных.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # Разделяем данные на подвыборки
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

## Извлекаем признаки
Для каждого изображения мы рассчитаем гистограмму ориентированных градиентов (HOG), а также цветовую гистограмму, используя каналы оттенков в цветовом пространстве HSV. Мы будем формировать окончательный вектор признаков для каждого изображения, объединяя векторы признаков HOG и цветовой гистограммы.

Гистограмма HOG должна захватывать текстуру изображения, игнорируя информацию о цвете, а цветовая гистограмма - наоборот. 

Функции `hog_feature` и` color_histogram_hsv` работают с одним изображением и возвращают для него вектор признаков. Функция `extract_features` берет набор изображений и функции признаков, а затем оценивает каждую функцию признаков на каждом изображении, сохраняя результаты в матрице, где каждый столбец представляет собой объединение всех векторов признаков для одного изображения.

In [ ]:
from cs231n.features import *

num_color_bins = 10 # Число бинов в гистограмме
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Предварительная обработка: вычитаем среднее значение
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Делим на стандартное отклонение. Это гарантирует, что масштаб всех
# признаков будет примерно одинаковым.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Добавляем смещение
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

## Обучаем SVM на признаках
Используя код для многоклассового SVM, реализованный ранее, обучите SVM на извлечённых выше признаках.
Вы должны получить лучшие результаты, чем для обычных пикселей.

In [ ]:
# Используем оценочную выборку для настройки скорости обучения и силы регуляризации

from cs231n.classifiers.linear_classifier import LinearSVM

learning_rates = [1e-9, 1e-8, 1e-7]
regularization_strengths = [5e4, 5e5, 5e6]

results = {}
best_val = -1
best_svm = None

################################################################################
# TODO:                                                                        #
# Используйте оценочную выборку, чтобы настроить гиперпараметры (силу          # 
# регялризации и скорость обучения). Сохраните лучший классификатор в          #
# переменную best_svm. Вы также можете попробовать помеять число бинов.        #
# Поэкспереминтируйте с различными значениями; вы должны получить точность     #
# классификации около 0.35 на оценочной выборке.                               #
################################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****

# Выводим результаты
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('Скорость обучения: %e регуляризация: %e точность обучения: %f точность оценки: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('Лучшая точность, достигнутая в результате кросс-валидации: %f' % best_val)

In [ ]:
# Оцениваем SVM на тестовой выборке
y_test_pred = best_svm.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

In [ ]:
# Попробуйте визуализировать результаты и посмотреть, какие ошибки допускает ваш классификатор.

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    idxs = np.random.choice(idxs, examples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

## Обучаем нейросеть на признаках
Для полноты картины попробуем обучить ранее реализованную двухслойную нейросеть на признаках изображения.

In [ ]:
# Предварительная обработка: убираем смещение
# Убедитесь, что запускаете это ячейку только ОДИН раз
print(X_train_feats.shape)
X_train_feats = X_train_feats[:, :-1]
X_val_feats = X_val_feats[:, :-1]
X_test_feats = X_test_feats[:, :-1]

print(X_train_feats.shape)

In [ ]:
from cs231n.classifiers.neural_net import TwoLayerNet

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO: Обучите двухслойную нейросеть на признаках изображения. Вы можете      #
# попробовать разные параметры кросс-валидации. Сохраните лучшую модель в      #
# переменную best_net variable.                                                #
################################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****


In [ ]:
# Запустите лучшую модель на тестовой выборке. Вам необходимо достигнуть
# точности более чем 55%.

test_acc = (best_net.predict(X_test_feats) == y_test).mean()
print(test_acc)